# How to debug your LLM apps

Like building any type of software, at some point you'll need to debug when building with LLMs. A model call will fail, or model output will be misformatted, or there will be some nested model calls and it won't be clear where along the way an incorrect output was created.

There are three main methods for debugging:

- Verbose Mode: This adds print statements for "important" events in your chain.
- Debug Mode: This add logging statements for ALL events in your chain.
- LangSmith Tracing: This logs events to [LangSmith](https://docs.smith.langchain.com/) to allow for visualization there.

|                        | Verbose Mode | Debug Mode | LangSmith Tracing |
|------------------------|--------------|------------|-------------------|
| Free                   | ✅            | ✅          | ✅                 |
| UI                     | ❌            | ❌          | ✅                 |
| Persisted              | ❌            | ❌          | ✅                 |
| See all events         | ❌            | ✅          | ✅                 |
| See "important" events | ✅            | ❌          | ✅                 |
| Runs Locally           | ✅            | ✅          | ❌                 |


## Tracing

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

Let's suppose we have an agent, and want to visualize the actions it takes and tool outputs it receives. Without any debugging, here's what we see:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>
```

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

tools = [TavilySearchResults(max_results=1)]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan.\n\nChristopher Nolan was born on July 30, 1970. To calculate his age in days as of today:\n\n1. First, find the total number of days from his birthdate to today\'s date.\n2. Today\'s date is December 7, 2023.\n\nUsing the formula to calculate the number of days between two dates:\n\n\\[ \\text{Age in days} = (\\text{Current Year} - \\text{Birth Year}) \\times 365 + (\\text{Days for leap years}) + (\\text{Days from birth month to current month}) \\]\n\nCalculating the leap years between 1970 and 2023, and considering the days from July 30 to December 7:\n\n- From 1970 to 2023, there are approximately 53 years.\n- Leap years between 1970 and 2023 (every 4 years, except century years not divisible by 400): 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020. That\'s 13 leap years.\n- Days from July 30 to Dece

We don't get much output, but since we set up LangSmith we can easily see what happened under the hood:

https://smith.langchain.com/public/a89ff88f-9ddc-4757-a395-3a1b365655bf/r

## `set_debug` and `set_verbose`

If you're prototyping in Jupyter Notebooks or running Python scripts, it can be helpful to print out the intermediate steps of a chain run.

There are a number of ways to enable printing at varying degrees of verbosity.

Note: These still work even with LangSmith enabled, so you can have both turned on and running at the same time


### `set_verbose(True)`

Setting the `verbose` flag will print out inputs and outputs in a slightly more readable format and will skip logging certain raw outputs (like the token usage stats for an LLM call) so that you can focus on application logic.

In [3]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)



> Entering new AgentExecutor chain...



Invoking: `tavily_search_results_json` with `{'query': 'director of the 2023 film Oppenheimer'}`




[{'url': 'https://www.imdb.com/title/tt15398776/fullcredits/', 'content': 'Oppenheimer (2023) cast and crew credits, including actors, actresses, directors, writers and more. Menu. ... director of photography: behind-the-scenes Jason Gary ... best boy grip ... film loader Luc Poullain ... aerial coordinator'}]
Invoking: `tavily_search_results_json` with `{'query': 'birth date of Christopher Nolan'}`




[{'url': 'https://www.imdb.com/name/nm0634240/bio/', 'content': 'Christopher Nolan. Writer: Tenet. Best known for his cerebral, often nonlinear, storytelling, acclaimed Academy Award winner writer/director/producer Sir Christopher Nolan CBE was born in London, England. Over the course of more than 25 years of filmmaking, Nolan has gone from low-budget independent films to working on some of the biggest blockbusters ever made and became one of the most ...'}]

The 2023 film "Oppenheimer" was directed by Christopher Nolan.

Christopher Nolan was born on July 30, 1970. To calculate his age in days as of today:

1. First, find the total number of days from his birthdate to today.
2. Today's date is December 7, 2023.

Using the formula for calculating the number of days between two dates:

\[ \text{Age in days} = (\text{Current Year} - \text{Birth Year}) \times 365 + \text{Days for leap years} + \text{Days from birthday this year to today} \]

Calculating the leap years between 1970 and 2023, and considering that 1972, 1976, ..., 2020 are leap years, we get approximately 13 leap years.

Christopher Nolan's birthday this year would have been July 30, 2023. From July 30 to December 7 is 130 days.

\[ \text{Age in days} = (2023 - 1970) \times 365 + 13 + 130 = 53 \times 365 + 13 + 130 = 19345 + 13 + 130 = 19488 \text{ days} \]

Therefore, Christopher Nolan is approximately 19,488 days old as of December 7, 2023.

> Finished chain.


{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan.\n\nChristopher Nolan was born on July 30, 1970. To calculate his age in days as of today:\n\n1. First, find the total number of days from his birthdate to today.\n2. Today\'s date is December 7, 2023.\n\nUsing the formula for calculating the number of days between two dates:\n\n\\[ \\text{Age in days} = (\\text{Current Year} - \\text{Birth Year}) \\times 365 + \\text{Days for leap years} + \\text{Days from birthday this year to today} \\]\n\nCalculating the leap years between 1970 and 2023, and considering that 1972, 1976, ..., 2020 are leap years, we get approximately 13 leap years.\n\nChristopher Nolan\'s birthday this year would have been July 30, 2023. From July 30 to December 7 is 130 days.\n\n\\[ \\text{Age in days} = (2023 - 1970) \\times 365 + 13 + 130 = 53 \\times 365 + 13 + 130 = 19345 + 13 + 130 = 19488 \\text{ days} \\

### `set_debug(True)`

Setting the global `debug` flag will cause all LangChain components with callback support (chains, models, agents, tools, retrievers) to print the inputs they receive and outputs they generate. This is the most verbose setting and will fully log raw inputs and outputs.

In [4]:
from langchain.globals import set_debug

set_debug(True)
set_verbose(False)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

[llm/end] [chain:AgentExecutor > chain:RunnableSequence > llm:ChatOpenAI] [2.85s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "model_name": "gpt-4-turbo-2024-04-09",
          "system_fingerprint": "fp_1f2231a495"
        },
        "type": "ChatGenerationChunk",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessageChunk"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "tool_calls": [
                {
                  "index": 0,
                  "id": "call_wUHXKdgLCbumDN7LqvRLQk5e",
                  "function": {
                    "arguments": "{\"query\": \"director of the 2023 film Oppenheimer\"}",
                    "name": "tavily_search_results_json"
             

[tool/end] [chain:AgentExecutor > tool:tavily_search_results_json] [3.13s] Exiting Tool run with output:
"[{'url': 'https://www.imdb.com/title/tt15398776/fullcredits/', 'content': 'Oppenheimer (2023) cast and crew credits, including actors, actresses, directors, writers and more. Menu. ... director of photography: behind-the-scenes Jason Gary ... best boy grip ... film loader Luc Poullain ... aerial coordinator'}]"
[tool/start] [chain:AgentExecutor > tool:tavily_search_results_json] Entering Tool run with input:
"{'query': 'Christopher Nolan birthdate'}"


[tool/end] [chain:AgentExecutor > tool:tavily_search_results_json] [3.14s] Exiting Tool run with output:
"[{'url': 'https://www.imdb.com/name/nm0634240/bio/', 'content': 'Christopher Nolan. Writer: Tenet. Best known for his cerebral, often nonlinear, storytelling, acclaimed Academy Award winner writer/director/producer Sir Christopher Nolan CBE was born in London, England. Over the course of more than 25 years of filmmaking, Nolan has gone from low-budget independent films to working on some of the biggest blockbusters ever made and became one of the most ...'}]"
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with 

[llm/end] [chain:AgentExecutor > chain:RunnableSequence > llm:ChatOpenAI] [11.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The 2023 film \"Oppenheimer\" was directed by Christopher Nolan.\n\nChristopher Nolan was born on July 30, 1970. To calculate his age in days as of today (December 7, 2023):\n\n1. Calculate the total number of days from his birthdate to today.\n2. Use the formula: \\(\\text{Age in days} = \\text{Total days from birthdate to today}\\).\n\nLet's calculate:\n\n- From July 30, 1970, to July 30, 2023, is 53 years.\n- From July 30, 2023, to December 7, 2023, is 130 days.\n\nTotal days = \\(53 \\times 365 + 130 + \\text{leap year days}\\).\n\nLeap year days from 1970 to 2023 (1972, 1976, ..., 2020) = 14 days.\n\nSo, total days = \\(19345 + 130 + 14 = 19489\\) days.\n\nChristopher Nolan is 19,489 days old as of December 7, 2023.",
        "generation_info": {
          "finish_reason": "stop",
          "model_name": "gpt-4-turbo-20

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan.\n\nChristopher Nolan was born on July 30, 1970. To calculate his age in days as of today (December 7, 2023):\n\n1. Calculate the total number of days from his birthdate to today.\n2. Use the formula: \\(\\text{Age in days} = \\text{Total days from birthdate to today}\\).\n\nLet\'s calculate:\n\n- From July 30, 1970, to July 30, 2023, is 53 years.\n- From July 30, 2023, to December 7, 2023, is 130 days.\n\nTotal days = \\(53 \\times 365 + 130 + \\text{leap year days}\\).\n\nLeap year days from 1970 to 2023 (1972, 1976, ..., 2020) = 14 days.\n\nSo, total days = \\(19345 + 130 + 14 = 19489\\) days.\n\nChristopher Nolan is 19,489 days old as of December 7, 2023.'}